In [1]:
# -*- coding: utf-8 -*-
import csv
from ckiptagger import WS
from opencc import OpenCC
import re
import gensim

cc = OpenCC('s2tw')
ws = WS("./data")

chi_stopWords=[]
with open('stopWords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        chi_stopWords.append(data)

c:\Users\Sam\Desktop\graduate_courses_data\Information_Retrieval\final_project\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\Sam\Desktop\graduate_courses_data\Information_Retrieval\final_project\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\Sam\Desktop\graduate_courses_data\Information_Retrieval\final_project\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, 

In [2]:
def preprocess(text):
    chi_tokens = []
    chi_text = "".join(re.compile(r'[\u4e00-\u9fa5]').findall(text))
    if len(chi_text) > 0:
        chi_text_seg = ws([chi_text])[0]
        chi_tokens = list(filter(lambda a: a not in chi_stopWords, chi_text_seg))

    eng_tokens = []
    eng_text = " ".join(re.compile(r'[\u0061-\u007a]+').findall(text.lower()))
    if len(eng_text) > 0:
        eng_text = gensim.parsing.remove_stopwords(eng_text)
        eng_tokens = list(gensim.utils.tokenize(eng_text))

    tokens = chi_tokens + eng_tokens
    return tokens

In [3]:
class BookmarkFullTextCorpus():
    def __init__(self, archive_path):
        self.path = archive_path
        self.dictionary = gensim.corpora.Dictionary(self.iter_texts())
        
    def iter_texts(self):
        with open(self.path, encoding='utf-8') as f:
            for row in csv.DictReader(f):
                yield preprocess(row["plain_text"])
                
    def __iter__(self):
        for document in self.iter_texts():
            yield self.dictionary.doc2bow(document)

    def __len__(self):
        return self.dictionary.num_docs
    
    def get_original(self, key):
        with open('data_sample.csv', encoding='utf-8') as f:
            for i, row in enumerate(csv.DictReader(f)):
                if i == key:
                    return row

In [4]:
content = BookmarkFullTextCorpus('data_sample.csv')

index = gensim.similarities.Similarity('data_sample.csv',
                                       content, 
                                       num_features=len(content.dictionary),
                                       num_best=15)

In [5]:
def search(query):
    query_bow = content.dictionary.doc2bow(preprocess(query))
    
    for doc, percent in index[query_bow]:
        result = content.get_original(doc)
        print("{:.5f} => {} ({})".format(percent, result['title'], result['url']))

In [6]:
search('深度學習')

0.41304 => 英雄集結：深度學習的魔法使們 (https://ithelp.ithome.com.tw/users/20112540/ironman/2064?sc=iThelpR)
0.41304 => 超簡單用Python預測股價 | FinLab 量化實驗室 (https://www.finlab.tw/%E8%B6%85%E7%B0%A1%E5%96%AE-Machine-Learning-%E9%A0%90%E6%B8%AC%E8%82%A1%E5%83%B9/)
0.06483 => [電腦視覺] 如何應用 Cloud AutoML Vision 辨識怪盜基德、工藤新一、服部平次 – 沒一村生活點滴 (https://noootown.wordpress.com/2018/07/25/apply-cloud-automl-vision-distinguish-konan/)
0.01116 => 語言支援  |  Cloud Speech-to-Text API  |  Google Cloud (https://cloud.google.com/speech-to-text/docs/languages?hl=zh-TW)


In [7]:
lsi = gensim.models.LsiModel(content,
                             num_topics=100,
                             power_iters=10,
                             id2word=content.dictionary)

In [8]:
lsi_index = gensim.similarities.Similarity('data_sample.csv',
                                           lsi[content],
                                           num_features=lsi.num_topics,
                                           num_best=15)

In [9]:
def lsi_search(query):
    query_bow = content.dictionary.doc2bow(preprocess(query))
    
    for doc, percent in lsi_index[lsi[query_bow]]:
        result = content.get_original(doc)
        print("{:.5f} => {} ({})".format(percent, result['title'], result['url']))

In [10]:
lsi_search('深度學習')

0.99641 => 英雄集結：深度學習的魔法使們 (https://ithelp.ithome.com.tw/users/20112540/ironman/2064?sc=iThelpR)
0.99641 => 超簡單用Python預測股價 | FinLab 量化實驗室 (https://www.finlab.tw/%E8%B6%85%E7%B0%A1%E5%96%AE-Machine-Learning-%E9%A0%90%E6%B8%AC%E8%82%A1%E5%83%B9/)
0.15640 => [電腦視覺] 如何應用 Cloud AutoML Vision 辨識怪盜基德、工藤新一、服部平次 – 沒一村生活點滴 (https://noootown.wordpress.com/2018/07/25/apply-cloud-automl-vision-distinguish-konan/)
0.02692 => 語言支援  |  Cloud Speech-to-Text API  |  Google Cloud (https://cloud.google.com/speech-to-text/docs/languages?hl=zh-TW)
0.00000 => A Game-Engine-Based Learning Environment Framework for Artificial General Intelligence | SpringerLink (https://link.springer.com/chapter/10.1007/978-3-319-46687-3_39)
0.00000 => Whole brain connectomic architecture to develop general artificial intelligence - ScienceDirect (https://www.sciencedirect.com/science/article/pii/S1877050918300498)
0.00000 => ShareCourse 學聯網 (https://www.sharecourse.net/sharecourse/course/content/homepage/1700)
0.00000 => 不到 2 

In [11]:
lsi.print_topics()

[(0,
  '0.312*"model" + 0.309*"test" + 0.215*"images" + 0.211*"style" + 0.200*"horse" + 0.169*"zebra" + 0.158*"trained" + 0.153*"data" + 0.148*"datasets" + 0.138*"download"'),
 (1,
  '-0.295*"學習" + -0.292*"like" + -0.253*"莉森" + -0.253*"揪" + -0.253*"深度" + -0.229*"留言" + -0.196*"文章" + -0.176*"瀏覽" + -0.174*"魔法" + -0.167*"分享"'),
 (2,
  '0.523*"聊天" + 0.408*"机器人" + 0.306*"个" + 0.166*"做" + 0.140*"们" + 0.127*"一个" + 0.102*"为" + 0.089*"户" + 0.089*"时间" + -0.082*"like"'),
 (3,
  '0.412*"python" + 0.384*"hours" + 0.295*"course" + 0.207*"starfilled" + 0.192*"videos" + 0.169*"m" + 0.162*"programming" + 0.148*"everybody" + 0.147*"reading" + 0.133*"complete"'),
 (4,
  '-0.371*"es" + -0.355*"espa" + -0.324*"ol" + -0.278*"英文" + -0.247*"ar" + -0.247*"en" + -0.232*"english" + -0.170*"西班牙" + -0.159*"語言" + -0.139*"印度"'),
 (5,
  '0.390*"cloud" + 0.275*"訓練" + 0.256*"automl" + 0.201*"雜筆" + 0.183*"資料" + 0.160*"圖片" + 0.144*"google" + 0.143*"使用" + 0.124*"vision" + 0.110*"vcm"'),
 (6,
  '-0.361*"learning" + -0.208*"